# Agent: Weather (Phase 3)

This notebook runs a dedicated weather agent that publishes weather state and tick events over MQTT.

In [ ]:
# Cell purpose: import project helpers and load config for MQTT connection.
from __future__ import annotations

import json
import random
import time

from simulated_city.config import load_config
from simulated_city.mqtt import MqttConnector, MqttPublisher
from simulated_city.payloads import build_weather_payload
from simulated_city.topics import weather_state_topic, weather_tick_topic

config = load_config()
print(f"MQTT broker: {config.mqtt.host}:{config.mqtt.port} (tls={config.mqtt.tls})")

In [ ]:
# Cell purpose: configure weather publishing cadence and connect MQTT.
rng = random.Random((config.simulation.seed if config.simulation and config.simulation.seed is not None else 42) + 1000)

WEATHER_INTERVAL_MIN_S = 10
WEATHER_INTERVAL_MAX_S = 15
MAX_PUBLISH_CYCLES = 10

state_topic = weather_state_topic()
tick_topic = weather_tick_topic()

connector = MqttConnector(config.mqtt, client_id_suffix="weather")
connector.connect()
if not connector.wait_for_connection(timeout=10.0):
    raise TimeoutError("Weather agent could not connect to MQTT within timeout")
publisher = MqttPublisher(connector)

print(f"Publishing weather state to: {state_topic}")
print(f"Publishing weather tick to:  {tick_topic}")

In [ ]:
# Cell purpose: run a weather loop that alternates sunny/rain and publishes state + tick payloads.
weather_state = "sunny"
tick = 0

try:
    for cycle in range(1, MAX_PUBLISH_CYCLES + 1):
        state_payload = build_weather_payload(weather_state=weather_state, tick=tick)
        tick_payload = build_weather_payload(weather_state=weather_state, tick=tick)

        publisher.publish_json(state_topic, json.dumps(state_payload), qos=1, retain=True)
        publisher.publish_json(tick_topic, json.dumps(tick_payload), qos=0, retain=False)

        print(
            f"cycle={cycle} tick={tick} state={weather_state} "
            f"interval={WEATHER_INTERVAL_MIN_S}-{WEATHER_INTERVAL_MAX_S}s"
        )

        wait_s = rng.randint(WEATHER_INTERVAL_MIN_S, WEATHER_INTERVAL_MAX_S)
        time.sleep(wait_s)

        weather_state = "rain" if weather_state == "sunny" else "sunny"
        tick += 1
finally:
    connector.disconnect()
    print("Weather agent disconnected from MQTT.")